In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

###Combining Filters
Like factors, filters can be combined. Combining filters is done using the `&` (and) and `|` (or) operators. For example, let's say we want to screen for securities that are in the top 10% of average dollar volume and have a latest close price above $20. To start, let's make a high dollar volume filter using an `AverageDollarVolume` factor and `percentile_between`:

In [2]:
dollar_volume = AverageDollarVolume(window_length=30)
high_dollar_volume = dollar_volume.percentile_between(90, 100)

Note: `percentile_between` is a `Factor` method returning a `Filter`.

Next, let's create a `latest_close` factor and define a filter for securities that closed above $20:

In [3]:
latest_close = USEquityPricing.close.latest
above_20 = latest_close > 20

Now we can combine our `high_dollar_volume` filter with our `above_20` filter using the `&` operator:

In [4]:
tradeable_filter = high_dollar_volume & above_20

This filter will evaluate to `True` for securities where both `high_dollar_volume` and `above_20` are `True`. Otherwise, it will evaluate to `False`. A similar computation can be made with the `|` (or) operator.

If we want to use this filter as a screen in our pipeline, we can simply pass `tradeable_filter` as the `screen` argument.

In [5]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = dollar_volume.percentile_between(90, 100)

    latest_close = USEquityPricing.close.latest
    above_20 = latest_close > 20

    tradeable_filter = high_dollar_volume & above_20

    return Pipeline(
        columns={
            'percent_difference': percent_difference
        },
        screen=tradeable_filter
    )

When we run this, our pipeline output now only includes ~700 securities.

In [6]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
print 'Number of securities that passed the filter: %d' % len(result)
result

Number of securities that passed the filter: 737


percent_difference
2015-05-05 00:00:00+00:00 Equity(24 [AAPL])               0.016905
                          Equity(62 [ABT])                0.014385
                          Equity(67 [ADSK])              -0.003921
                          Equity(76 [TAP])               -0.008759
                          Equity(114 [ADBE])              0.009499
                          Equity(122 [ADI])               0.009271
                          Equity(128 [ADM])               0.015760
                          Equity(154 [AEM])               0.026035
                          Equity(161 [AEP])               0.010405
                          Equity(168 [AET])               0.005853
                          Equity(185 [AFL])              -0.002239
                          Equity(216 [HES])               0.036528
                          Equity(239 [AIG])               0.012322
                          Equity(270 [AKRX])             -0.024963
                          Equity(300 [ALK])               0.015147
                          Equity(301 [ALKS])             -0.033228
                          Equity(328 [ALTR])              0.012284
                          Equity(357 [TWX])               0.000365
                          Equity(368 [AMGN])              0.008860
                          Equity(438 [AON])               0.002327
                          Equity(448 [APA])               0.035926
                          Equity(455 [APC])               0.049153
                          Equity(460 [APD])              -0.006999
                          Equity(624 [ATW])               0.014957
                          Equity(630 [ADP])              -0.002134
                          Equity(679 [AXP])              -0.011809
                          Equity(693 [AZO])               0.002395
                          Equity(698 [BA])               -0.016685
                          Equity(734 [BAX])               0.009414
                          Equity(739 [BBBY])             -0.027796
...                                                            ...
                          Equity(45269 [EVHC])           -0.004877
                          Equity(45451 [FEYE])            0.042108
                          Equity(45558 [BURL])           -0.053654
                          Equity(45570 [JNUG])            0.053977
                          Equity(45618 [AR])              0.091085
                          Equity(45769 [WUBA])            0.234141
                          Equity(45804 [ASHR])            0.082573
                          Equity(45815 [TWTR])           -0.077268
                          Equity(45971 [AAL])             0.008087
                          Equity(45978 [ATHM])            0.063568
                          Equity(45993 [HLT])            -0.000895
                          Equity(46015 [ALLY])            0.009605
                          Equity(46308 [ASPX])            0.054145
                          Equity(46631 [GOOG])            0.004730
                          Equity(46693 [GRUB])           -0.016904
                          Equity(46979 [JD])              0.058362
                          Equity(47169 [KITE])           -0.049366
                          Equity(47208 [GPRO])            0.061078
                          Equity(47230 [NSAM])           -0.037879
                          Equity(47430 [MBLY])            0.050288
                          Equity(47740 [BABA])           -0.008354
                          Equity(47777 [CFG])             0.025703
                          Equity(47779 [CYBR])            0.101844
                          Equity(48065 [AXTA])            0.034600
                          Equity(48317 [JUNO])           -0.103370
                          Equity(48384 [QRVO])           -0.050578
                          Equity(48892 [IGT])             0.005591
                          Equity(48934 [ETSY])           -0.030142
                          Eq

In the next lesson, we'll look at masking factors and filters.